# Neural Network Exercise 5 (Elman & Jordan Networks)

Amirkabir University of Technology

Dr. Safabakhsh

By Gholamreza Dar 400131018

Spring 2022

## Imports

In [ ]:
import os
import datetime
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
sns.set_style('dark')

import tensorflow as tf
from sklearn.model_selection import train_test_split

## Functions

### Data loading

In [ ]:
def convert_label_to_binary(label):
    if label == 'normal' or label == 'ok':
        return 1
    else:
        return 0

In [ ]:
def extract_data(file_path):
    '''Parse the data from 'file_path' and store the number lines in the segment list and the label lines in the labels list'''
    segments = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f.readlines():
            # Number line
            if line.startswith("\t"):
                segments.append(list(map(int, line.split())))
            # Whitespace
            elif line.startswith("\n"):
                pass
            # Label line
            else:
                labels.append(convert_label_to_binary(line[:-1]))

    return segments, labels


In [ ]:
def load_data():
    '''Load the data from all 5 files and return a (463, 15, 6) numpy array and labels as a (463,) numpy array'''

    base_dir = "data"
    file_names = ["lp1.data", "lp2.data", "lp3.data", "lp4.data", "lp5.data"]
    
    X = []
    y = []

    # Loop through the files and extract the data and concatenate them
    for file_name in file_names:
        segments, labels = extract_data(os.path.join(base_dir, file_name))
        X += segments
        y += labels
    
    # Convert lists to numpy array and reshape them
    X = np.array(X)
    X = X.reshape(-1, 15, 6)
    y = np.array(y)
    
    return X, y

### Elman network

In [ ]:
class ElmanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(ElmanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            state_t = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))

        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))
        return y

    def evaluate(self, x, y):
        y_pred = self(x)
        return tf.keras.metrics.binary_accuracy(y, y_pred)

### Jordan Network

In [ ]:
class JordanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(JordanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y_intermediate = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y_intermediate = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            hidden = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))
            y_intermediate = tf.keras.activations.tanh(tf.matmul(self.W_y_intermediate, tf.reshape(hidden, (self.hidden_units,1))) + tf.reshape(self.b_y_intermediate, (self.hidden_units,1)))
            state_t = y_intermediate
        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))

        return y
    
    def evaluate(self, x, y):
        y_pred = self(x)
        return tf.keras.metrics.binary_accuracy(y, y_pred)

### Experiment functions

In [ ]:
def train_elman(hidden_count, epochs=30, n_classes=2, lr=1e-3):
    elman_nn = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    elman_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    elman_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    test_score = elman_nn.evaluate(X_test, y_test)

    return elman_nn, test_score

In [ ]:
def train_jordan(hidden_count, epochs=30, n_classes=2, lr=1e-3):
    jordan_nn = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    jordan_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    test_score = jordan_nn.evaluate(X_test, y_test)

    return jordan_nn, test_score

### Ensemble Functions

In [ ]:
def create_elman_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Elman submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        elman_submodel = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        elman_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=elman_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [ ]:
def create_jordan_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Jordan submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        jordan_submodel = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        jordan_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=jordan_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [ ]:
def create_ensemble_model(sub_models, active_index=None):
    '''This function takes a list of sub-models and returns an ensemble of them.
    Works with elman and jordan sub-models(and any combination of them).'''

    # We have to rename the layers to avoid duplicate layer names in the ensemble model
    for i, sub_model in enumerate(sub_models):
        for layer in sub_model.layers:
            layer.name = f'ensemble_{i+1}_{layer.name}'

    # Freeze every submodel except for the 'active_index' one(if 'active_index' is set)
    if active_index is not None:
        for i, sub_model in enumerate(sub_models):
            if i != active_index:
                for layer in sub_model.layers:
                    layer.trainable = False
            else:
                for layer in sub_model.layers:
                    layer.trainable = True
        
    # Create the ensemble model
    inputs = [sub_model.input for sub_model in sub_models]
    outputs = [sub_model.output for sub_model in sub_models]
    concat = tf.keras.layers.Concatenate(outputs)
    ensemble_output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)
    ensemble_output = tf.keras.layers.Flatten()(ensemble_output)
    ensemble_model = tf.keras.Model(inputs=inputs, outputs=ensemble_output)

    # Compile the ensemble model
    ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])

    return ensemble_model

### Ensemble Experiment Functions


In [ ]:
def elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates elman sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = create_elman_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(elman_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(elman_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(elman_submodels))]
    X_test_ensemble = [X_test for i in range(len(elman_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [ ]:
def jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    jordan_submodels = create_jordan_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(jordan_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(jordan_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(jordan_submodels))]
    X_test_ensemble = [X_test for i in range(len(jordan_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [ ]:
def endgame_ensemble_experiment(elman_hidden_count_list=[10, 25, 50], jordan_hidden_count_list=[10, 25, 50]):
    '''Creates elman and jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = []
    if len(elman_hidden_count_list)>0:
        elman_submodels = create_elman_submodels(elman_hidden_count_list)

    jordan_submodels = []
    if len(jordan_hidden_count_list)>0:
        jordan_submodels = create_jordan_submodels(jordan_hidden_count_list)

    # Total count of sub-models
    total_submodels = len(elman_submodels) + len(jordan_submodels)

    # Split X_train for each sub-model 
    X_train_list = []
    for i in range(total_submodels):
        X_train_current = X_train[int(i*X_train.shape[0]/total_submodels):int((i+1)*X_train.shape[0]/total_submodels)]
        X_train_list.append(X_train_current)

    # Choose X_train and copy X_validation for all of the sub-models
    X_train_ensemble = [X_train_list[i] for i in range(total_submodels)]
    X_validation_ensemble = [X_validation for i in range(total_submodels)]
    X_test_ensemble = [X_test for i in range(total_submodels)]
    
    for i in range(total_submodels):
        # Create the mixed ensemble model and freeze every submodel except the i-th one
        ensemble_model = create_ensemble_model(elman_submodels + jordan_submodels, active_index=i)

        # Train the ensemble model
        ensemble_model.fit(
            X_train_ensemble,
            y_train,
            batch_size=1,
            epochs=30,
            validation_data=(X_validation_ensemble, y_validation),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/endgame_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

## Main

### Loading the data

In [ ]:
X, y = load_data()
print("X.shape:", X.shape)
print("y.shape:", y.shape)

### Train, Test, validation split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.125, random_state=1)

### Normalize Dataset?

In [ ]:
# Normalize the data

### Elman

In [ ]:
elman_nn = ElmanModel(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

elman_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=30,
    validation_data=(X_validation, y_validation),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

In [ ]:
elman_nn.evaluate(X_test, y_test)

### Jordan

In [ ]:
jordan_nn = JordanModel(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])

jordan_nn.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=30,
    validation_data=(X_validation, y_validation),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

In [ ]:
jordan_nn.evaluate(X_test, y_test)

### Experiments

In [ ]:
# Experiment with different hidden layer counts
elman_10, elman_10_score = train_elman(10)
elman_25, elman_25_score = train_elman(25)
elman_50, elman_50_score = train_elman(50)

In [ ]:
print("elman_10_score:", elman_10_score)
print("elman_25_score:", elman_25_score)
print("elman_50_score:", elman_50_score)

In [ ]:
jordan_10, jordan_10_score = train_jordan(10)
jordan_25, jordan_25_score = train_jordan(25)
jordan_50, jordan_50_score = train_jordan(50)

In [ ]:
print("jordan_10_score:", jordan_10_score)
print("jordan_25_score:", jordan_25_score)
print("jordan_50_score:", jordan_50_score)

### Ensemble

In [ ]:
def create_elman_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Elman submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        elman_submodel = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        elman_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=elman_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [ ]:
def create_jordan_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Jordan submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        jordan_submodel = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        jordan_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=jordan_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [ ]:
def create_ensemble_model(sub_models, active_index=None):
    '''This function takes a list of sub-models and returns an ensemble of them.
    Works with elman and jordan sub-models(and any combination of them).'''

    # We have to rename the layers to avoid duplicate layer names in the ensemble model
    for i, sub_model in enumerate(sub_models):
        for layer in sub_model.layers:
            layer.name = f'ensemble_{i+1}_{layer.name}'

    # Freeze every submodel except for the 'active_index' one(if 'active_index' is set)
    if active_index is not None:
        for i, sub_model in enumerate(sub_models):
            if i != active_index:
                for layer in sub_model.layers:
                    layer.trainable = False
            else:
                for layer in sub_model.layers:
                    layer.trainable = True
        
    # Create the ensemble model
    inputs = [sub_model.input for sub_model in sub_models]
    outputs = [sub_model.output for sub_model in sub_models]
    concat = tf.keras.layers.Concatenate(outputs)
    ensemble_output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)
    ensemble_output = tf.keras.layers.Flatten()(ensemble_output)
    ensemble_model = tf.keras.Model(inputs=inputs, outputs=ensemble_output)

    # Compile the ensemble model
    ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])

    return ensemble_model

In [ ]:
def elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates elman sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = create_elman_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(elman_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(elman_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(elman_submodels))]
    X_test_ensemble = [X_test for i in range(len(elman_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [ ]:
def jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    jordan_submodels = create_jordan_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(jordan_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(jordan_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(jordan_submodels))]
    X_test_ensemble = [X_test for i in range(len(jordan_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [ ]:
def endgame_ensemble_experiment(elman_hidden_count_list=[10, 25, 50], jordan_hidden_count_list=[10, 25, 50]):
    '''Creates elman and jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = []
    if len(elman_hidden_count_list)>0:
        elman_submodels = create_elman_submodels(elman_hidden_count_list)

    jordan_submodels = []
    if len(jordan_hidden_count_list)>0:
        jordan_submodels = create_jordan_submodels(jordan_hidden_count_list)

    # Total count of sub-models
    total_submodels = len(elman_submodels) + len(jordan_submodels)

    # Split X_train for each sub-model 
    X_train_list = []
    for i in range(total_submodels):
        X_train_current = X_train[int(i*X_train.shape[0]/total_submodels):int((i+1)*X_train.shape[0]/total_submodels)]
        X_train_list.append(X_train_current)

    # Choose X_train and copy X_validation for all of the sub-models
    X_train_ensemble = [X_train_list[i] for i in range(total_submodels)]
    X_validation_ensemble = [X_validation for i in range(total_submodels)]
    X_test_ensemble = [X_test for i in range(total_submodels)]
    
    for i in range(total_submodels):
        # Create the mixed ensemble model and freeze every submodel except the i-th one
        ensemble_model = create_ensemble_model(elman_submodels + jordan_submodels, active_index=i)

        # Train the ensemble model
        ensemble_model.fit(
            X_train_ensemble,
            y_train,
            batch_size=1,
            epochs=30,
            validation_data=(X_validation_ensemble, y_validation),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/endgame_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [ ]:
def evaluate_ensemble(ensemble_model, X_test, y_test):
    '''Evaluates the ensemble model on the test set.'''
    test_loss, test_accuracy = ensemble_model.evaluate(X_test, y_test)
    print('Test accuracy:', test_accuracy)

#### Elman Only 

#### Jordan Only

#### Mixed

## Tests

In [ ]:
# elman_nn1 = ElmanModel(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

# elman_nn1.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
# elman_nn1.fit(
#     X_train,
#     y_train,
#     batch_size=1,
#     epochs=30,
#     validation_data=(X_validation, y_validation),
#     callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman1-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])


In [ ]:
# elman_nn2 = ElmanModel(input_dim=15, hidden_units=20, feature_size=6, n_classes=1)
# elman_nn2.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
# elman_nn2.fit(
#     X_train,
#     y_train,
#     batch_size=1,
#     epochs=30,
#     validation_data=(X_validation, y_validation),
#     callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman2-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

In [ ]:
# id=6
# print(y_train[id])

In [ ]:
# elman_nn1(X_train[np.newaxis,id])

In [ ]:
# elman_nn2(X_train[np.newaxis,id])

## Tensorboard

In [ ]:
# load tensorboard extension
%load_ext tensorboard

# start tensorboard in the 'logs' directory
%tensorboard --logdir=logs/